In [ ]:
colab = True

if colab:
  !pip install datasets tokenizers transformers
  !git clone https://github.com/thibautvalour/NLP_Intent_classification.git
  %cd NLP_Intent_classification

In [ ]:
import torch
from datasets import load_dataset

from model import BertBiLSTM
from utils import train, test
from data_formating import make_dataloader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device :', device)

In [ ]:
# Define all parameters
args = {'dataset_percentage': '1%',
        'max_sentence': 16,
        'max_word': 32,
        'device': device,
        'num_class': 5,
        'class_weights': torch.tensor([0.25, 0.25, 0.25, 0.25, 0.]).to(device),
        'unfreezed_bert_layer': 3,
        'bert_output_size': 768,
        'lstm_hidden_dimension':768,
        'lstm_layers':2,
        'dropout': 0.5, 
        'lr': 1e-4, 
        'batch_size': 4,
        'epochs': 3,
}

In [ ]:
# Load data
dataset_train , dataset_test = load_dataset("silicone","dyda_da", split=[f"train[:{args['dataset_percentage']}]",
                                                                         f"test[:{args['dataset_percentage']}]"])
# Format data
train_loader, test_loader = make_dataloader(dataset_train, dataset_test, args)

## define Model

In [ ]:
model = BertBiLSTM(args).to(args['device'])

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
# Train the model
for epoch in range(args['epochs']):
    train_loss, train_acc = train(model, optimizer, epoch, train_loader)
    test_loss, test_acc = test(model, epoch, test_loader)
    print(f'Epoch {epoch + 1}/{args["epochs"]} - Train loss: {train_loss:.3f} - Test loss: {test_loss:.3f} - Train acc: {train_acc:.3f} - Test acc: {test_acc:.3f}')